In [2]:
import numpy as np
import pandas as pd 

In [3]:
data = pd.read_csv('../Datagathering/data.csv')
data.head()

,Region,Country,wind speed(m/s),Temp(C),SP(kPa),PRECTOTCORR,ASSD(kWh/m²/day),year,month
0,Americas,Dominican Republic,4.62,24.69,101.28,0.07,5.65,2000,1
1,Americas,Dominican Republic,4.08,25.28,101.18,0.08,5.83,2000,2
2,Americas,Dominican Republic,3.11,26.10,101.02,2.12,5.97,2000,3
3,Americas,Dominican Republic,2.95,27.35,100.76,0.63,6.89,2000,4
4,Americas,Dominican Republic,5.03,28.77,100.58,5.44,5.22,2000,5


In [4]:
numeric_features = data[['year ', 'month', 'Temp(C)', 'SP(kPa)', 'ASSD(kWh/m²/day)']]
categorical_features = data[['Region', 'Country']]

In [5]:
# data scaling
from sklearn.preprocessing import StandardScaler, OneHotEncoder
scaler = StandardScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

numeric_features_scaled = scaler.fit_transform(numeric_features)
numeric_features_df = pd.DataFrame(numeric_features_scaled, columns=numeric_features.columns)

categorical_features_encoded = ohe.fit_transform(categorical_features)
categorical_features_df = pd.DataFrame(categorical_features_encoded, columns=ohe.get_feature_names_out(categorical_features.columns))
# Combine the processed features
processed_data = pd.concat([numeric_features_df, categorical_features_df], axis=1)

In [6]:
# split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

x= processed_data
y= data['wind speed(m/s)']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [7]:
# model training
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

# model evaluation
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [8]:
print(mse , r2)

0.08912276382419444 0.9526722396730888


In [9]:
# Cross-validation to evaluate the model
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, x_train, y_train, cv=5, scoring='r2')
print("Cross-validated R² scores:", scores)
print("Mean R²:", scores.mean())

Cross-validated R² scores: [0.94808085 0.94882886 0.94764379 0.94821627 0.9457213 ]
Mean R²: 0.9476982139401324


In [10]:
# save the model
import joblib
joblib.dump(model, '../Models/wind_speed.pkl')

['../Models/wind_speed.pkl']

In [11]:
joblib.dump(ohe, '../models/encoder4.pkl')
joblib.dump(scaler, '../models/scaler4.pkl')

['../models/scaler4.pkl']